In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
data = pd.read_csv("/content/drive/MyDrive/ESAA_OB_HW/ESAA_Project/Credit_card/train.csv")
len_data = len(data)

df = pd.read_csv("/content/drive/MyDrive/ESAA_OB_HW/ESAA_Project/Credit_card/df2.csv")
data = df[:len_data]
newdata = df[len_data:]
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X = data.drop(["credit"],axis=1)
#y = data["credit"].astype("int")
y= data["credit"].astype(float).round().astype(int)
y = le.fit_transform(y)
newdata_X = newdata.drop(["credit"],axis=1)
X_train,X_test, y_train,y_test = train_test_split(X,y,test_size = 0.3,random_state=42)

<ipython-input-2-72077d0c0bc6>:6: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/ESAA_OB_HW/ESAA_Project/Credit_card/df2.csv")


# Train and Evaluate Multiple Models


In [ ]:
# 모델의 성능이 더 낮아 사용X

#from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()

# X_scaled_d = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# # RandomForestClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score
# RF1 = RandomForestClassifier(n_estimators=1000,random_state=42)
# RF1.fit(X_scaled_d, y_train)
# print("RandomForest: ",accuracy_score(y_test, RF.predict(X_test_scaled)))

In [ ]:
# RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
RF = RandomForestClassifier(n_estimators=1000,random_state=42)
RF.fit(X, y)
#print("RandomForest: ",accuracy_score(y_test, RF.predict(X_test)))

RandomForestClassifier(n_estimators=1000, random_state=42)

In [ ]:
# GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
GB = GradientBoostingClassifier(n_estimators=1000, random_state=42)
GB.fit(X, y)
# print("GradientBoosting: ",accuracy_score(y_test, GB.predict(X_test)))

GradientBoostingClassifier(n_estimators=1000, random_state=42)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [None, 2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
RF = RandomForestClassifier(random_state=42)
random_search = RandomizedSearchCV(estimator=RF, param_distributions=param_dist, n_iter=100, cv=5, scoring='accuracy', random_state=42)
random_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
125 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
125 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py", line 340, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ut

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
                   n_iter=100,
                   param_distributions={'max_depth': [None, 10, 20],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [None, 2, 5, 10],
                                        'n_estimators': [50, 100, 200]},
                   random_state=42, scoring='accuracy')

In [ ]:
random_search.best_params_

{'n_estimators': 200,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'max_depth': None}

In [ ]:
# VotingClassifier
from sklearn.ensemble import VotingClassifier
VT = VotingClassifier(estimators=[('RF', RF), ('GB', GB)], voting='soft', n_jobs=-1)
VT.fit(X, y)
#print("Voting: ",accuracy_score(y_test, VT.predict(X_test)))

VotingClassifier(estimators=[('RF',
                              RandomForestClassifier(n_estimators=1000,
                                                     random_state=42)),
                             ('GB',
                              GradientBoostingClassifier(n_estimators=1000,
                                                         random_state=42))],
                 n_jobs=-1, voting='soft')

In [ ]:
import numpy as np
pred = pd.DataFrame(VT.predict_proba(newdata_X), index=np.arange(26457,36457))
pred.to_csv('/content/drive/MyDrive/ESAA_OB_HW/ESAA_Project/Credit_card/VT.csv', index= True )

In [ ]:
# VotingClassifier1
from sklearn.ensemble import VotingClassifier
VT1 = VotingClassifier(estimators=[('RF', RF), ('GB', GB)], voting='soft', n_jobs=-1)
VT1.fit(X, y)
print("Voting: ",accuracy_score(y_test, VT1.predict(X_test)))

Voting:  0.8834719072814311


In [ ]:
import numpy as np
pred = pd.DataFrame(VT1.predict_proba(newdata_X), index=np.arange(26457,36457))
pred.to_csv('/content/drive/MyDrive/ESAA_OB_HW/ESAA_Project/Credit_card/VT1.csv', index= True )

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

params = {
    'learning_rate': 0.1,
    'max_depth': 5,
    'n_estimators': 1000}
model = xgb.XGBClassifier(**params)
model.fit(X, y)

XGBoost:  0.7947845804988662


In [ ]:
# VotingClassifier2
from sklearn.ensemble import VotingClassifier
VT2 = VotingClassifier(estimators=[('RF', RF), ('GB', GB), ('XGB', model)], voting='soft', n_jobs=-1)
VT2.fit(X, y)
#print("Voting: ",accuracy_score(y_test, VT2.predict(X_test)))

VotingClassifier(estimators=[('RF',
                              RandomForestClassifier(n_estimators=1000,
                                                     random_state=42)),
                             ('GB',
                              GradientBoostingClassifier(n_estimators=1000,
                                                         random_state=42)),
                             ('XGB',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categoric...
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.1, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=5,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=1000, n_jobs=None,
                                            num_parallel_tree=None,
                                            objective='multi:softprob', ...))],
                 n_jobs=-1, voting='soft')

In [ ]:
import numpy as np
pred = pd.DataFrame(VT2.predict_proba(newdata_X), index=np.arange(26457,36457))
pred.to_csv('/content/drive/MyDrive/ESAA_OB_HW/ESAA_Project/Credit_card/VT2.csv', index= True )

# Improve and Apply a Model on New Membership Dataset



In [ ]:
# RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
RF_Final = RandomForestClassifier(n_estimators=100, random_state=42)
RF_Final.fit(X, y)

# GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
GB_Final = GradientBoostingClassifier(n_estimators=100, random_state=42)
GB_Final.fit(X, y)

# VotingClassifier
from sklearn.ensemble import VotingClassifier
VT_Final = VotingClassifier(estimators=[('RF', RF_Final), ('GB', GB_Final)], voting='soft', n_jobs=-1)
VT_Final.fit(X, y)


pred = pd.DataFrame()
pred["predicted"]= VT_Final.predict(newdata_X)

pred.to_csv("/content/drive/MyDrive/PatternRecogntionAndMachineLearning/Assignment1_prediction_1929041_황시현.csv")